In [24]:
# Suppress FutureWarning
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import NearestNeighbors
from utils.fidatabase import TTR
from utils.fidatabase import RTT

import refinitiv.data as rd
from refinitiv.data.content import symbol_conversion

import pandas_market_calendars as mcal
import datetime as dt

import os
from pathlib import Path
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
from urllib.parse import urlencode
from string import ascii_uppercase as alc


In [25]:
#rd.open_session()

In [26]:


urlarguments = {
    'type': 'stock',
    'period': 'full',
    'ticker_range': 'A',
    'adjustment': 'adj_split',
    'timeframe': '5min',
    'userid': '3YhhBJCZrEi4lNMKlwmppQ'

}


configuration_dict = {
    1 : { 'timeframe': '5min', 'segments ': 40, 'load_file' : '5min_40seg', 'start_date' : '2020-01-01', 'end_date' : '2023-06-06'},
    2 : { 'timeframe': '5min', 'segments ': 40, 'load_file' : '5min_40seg', 'start_date' : '2020-01-01', 'end_date' : '2023-06-06'},
}


url_base = "https://firstratedata.com/api/data_file?"

In [27]:
#from tqdm import tqdm
import requests 
from random import random

def get_size(url):
    response = requests.head(url)
    size = int(response.headers['Content-Length'])
    return size

def download_file(url):
    local_filename = f"./tmp/archive{random()}.zip"
    chunk_size = 400000000 # size of 1 chunk to download 100000000 = 100 MB
    
    with requests.get(url, stream=True, allow_redirects=True) as r:
        r.raise_for_status()
        
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size = chunk_size): # Download a 100 mb chunk
                #if chunk: 
                f.write(chunk)
    return local_filename




In [28]:
download_data = False

if download_data:
   for i in alc:
      urlarguments['ticker_range'] = i
      url = url_base + urlencode(urlarguments)
      print(f"Getting: {url}")
      #resp = urlopen(url)
      # read zipfile
      #zipfile = ZipFile(BytesIO(resp.read()))
      tmp = download_file(url)  
      zipfile = ZipFile(tmp)
      # get the csv file name
      zipfile.extractall(path='./tickdata/')
      zipfile.close() 
      os.remove(tmp)
      

In [29]:
full_stock_list = pd.read_csv('full_stock_list_m.csv', index_col="Ticker")
#full_stock_list_short = pd.read_csv('short_stock_list.csv', index_col="Ticker")

russell_tickers = full_stock_list.index.tolist()
#russell_tickers = RTT( russell_tickers[0].tolist() )

nyse_cal = mcal.get_calendar('NYSE')

In [44]:

#first_ratetickers = pd.read_csv("https://firstratedata.com/api/ticker_listing?type=stock&userid=3YhhBJCZrEi4lNMKlwmppQ", index_col="Ticker", parse_dates=True) #,  infer_datetime_format=True)
first_ratetickers = pd.read_csv("first_ratetickers-list.csv", index_col="Ticker", parse_dates=True, ) #,  infer_datetime_format=True)
first_ratetickers = pd.read_csv("first_ratetickers-list.csv", index_col="Ticker", parse_dates=True, keep_default_na=False) #,  infer_datetime_format=True)

first_ratetickers.columns = ['Name','start_date','end_date']
first_ratetickers["start_date"] = pd.to_datetime(first_ratetickers["start_date"], format='%Y-%m-%d').dt.tz_localize('America/New_York')
first_ratetickers["end_date"] = pd.to_datetime(first_ratetickers["end_date"], format='%Y-%m-%d').dt.tz_localize('America/New_York')


#first_ratetickers.to_csv('first_ratetickers-list.csv')
#first_ratetickers = full_stock_list

#stock_data = {}

In [45]:

firstrate_lastdate = pd.Timestamp("2023-06-06").tz_localize('America/New_York')

def get_tickers_for_window( start_window, end_window ):
    ticker_out = first_ratetickers[ (first_ratetickers['start_date'] <= start_window) & (first_ratetickers['end_date'] >= end_window) ].index.tolist()
    return ticker_out

def load_stock_data( ticker_name ):

    if( ticker_name in stock_data.keys() ):
       return stock_data[ticker_name]


    ticker_validstart = first_ratetickers.loc[ticker_name]['start_date']
    ticker_validend = first_ratetickers.loc[ticker_name]['end_date']
    delisted=False
    
    if( ticker_validend < firstrate_lastdate ):
        delisted=True

    if(ticker_name.find("AINV") > 0):
        print("Breakf for me")

    if delisted:
        if( ticker_name.find("DELISTED") > 0 ):
            filename = Path(f"./tickdata/{ticker_name}_full_{urlarguments['timeframe']}_adjsplitdiv.txt")
            if os.path.exists(filename) == False:
                filename = Path(f"./tickdata/{ticker_name}_full_{urlarguments['timeframe']}_adjsplit.txt")
                if os.path.exists(filename) == False:
                        print(f"Delisted Ticker {ticker_name} not found")
                        return None

        else:
            filename = Path(f"./tickdata/{ticker_name}-DELISTED_full_{urlarguments['timeframe']}_adjsplitdiv.txt")
            if os.path.exists(filename) == False:
                filename = Path(f"./tickdata/{ticker_name}-DELISTED_full_{urlarguments['timeframe']}_adjsplit.txt")
                if os.path.exists(filename) == False:
                    filename = Path(f"./tickdata/{ticker_name}_full_{urlarguments['timeframe']}_adjsplit.txt")  
                    if os.path.exists(filename) == False:
                        print(f"Delisted Ticker {ticker_name} not found")
                        return None
                    

    else:
        filename = Path(f"./tickdata/{ticker_name}_full_{urlarguments['timeframe']}_adjsplit.txt")  

    if os.path.exists(filename) == False:
        print(f"File {filename} not found Delisted: {delisted}  ")
        return None
    

    #print(f"Loading {filename}")
    
    df = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True)

    df.columns = ['Date', 'Open', 'High', 'Low', ticker_name, 'Volume']
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df.index = df.index.tz_localize('America/New_York')

    df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)
 

    #
    # Normalize index to the extend possible
    #
    #
    window_cal= nyse_cal.schedule(start_date=ticker_validstart, end_date=ticker_validend)
    valid_days = mcal.date_range(window_cal, frequency='5M')

    #Clean up the index
    df = df.resample('5min').median()
    df = df[ df.index.isin(valid_days)]
    

    #15min linimit
    fill_limit = int( pd.Timedelta("15min") / pd.Timedelta(urlarguments['timeframe'] ))
    df.fillna(method='ffill', limit=fill_limit, inplace=True)        
     
    stock_data[ticker_name] = df
    
    return df



tlist_date = get_tickers_for_window( '2006-01-01', '2007-01-01' )
msft = load_stock_data( "NA" )
#display(msft[140:180])

 

In [47]:
# window_df = None
# td = None
# import gc

# gc.collect()


0

In [51]:
start_date = pd.Timestamp('2006-01-01')
end_date = pd.Timestamp('2023-06-06')

window_df = None

tickers_in_window = first_ratetickers.index.tolist()
tickers_in_window.sort()

current_letter = 'A'
sub_df = None

#Process only 'M' tickers_in_window
# tmp_list = tickers_in_window.copy()
# for ticker in tmp_list:
#     if( ticker[0] != 'M' ):
#         tickers_in_window.remove(ticker)

for ticker in tickers_in_window:
    if( ticker[0] != current_letter ):
        print(f"Processing letter change -- saving previous {current_letter}")
        current_letter = ticker[0]

        if(sub_df is not None):
            if window_df is None:
                window_df = sub_df
            else:
                window_df = pd.concat([window_df, sub_df], axis=1)
        
        sub_df = None
       

    print(f"Processing {ticker}")     
    td = load_stock_data( ticker )
    if( td is None ):
        print(f"{ticker} not found - continuing and skipping")
        continue

    if sub_df is None:
        sub_df = td
    else:
        sub_df = pd.concat([sub_df, td], axis=1)

if sub_df is not None:
    window_df = pd.concat([window_df, sub_df], axis=1)
    


display(window_df.info())


    # if( ticker in ticker_window_count.keys() ):
    #     ticker_window_count[ticker] += 1
    # else:
    #     ticker_window_count[ticker] = 1


Processing A
Processing AA
Processing AABA-DELISTED
Processing AACG
Processing AADI
Processing AAIC
Processing AAL
Processing AAMC
Processing AAME
Processing AAN
Processing AAOI
Processing AAON
Processing AAP
Processing AAPL
Processing AAT
Processing AATC
Processing AAU
Processing AAWW-DELISTED
Processing AB
Processing ABBNY
Processing ABBV
Processing ABC
Processing ABCB
Processing ABCL
Processing ABCM
Processing ABEO
Processing ABEV
Processing ABG
Processing ABI-DELISTED
Processing ABIO
Processing ABK-DELISTED
Processing ABM
Processing ABMD
Processing ABNB
Processing ABOS
Processing ABR
Processing ABS-DELISTED
Processing ABSI
Processing ABST
Processing ABT
Processing ABTX-DELISTED
Processing ABUS
Processing ABVC
Processing AC
Processing ACA
Processing ACAC
Processing ACAD
Processing ACAS-DELISTED
Processing ACAX
Processing ACB
Processing ACBI-DELISTED
Processing ACC-DELISTED
Processing ACCD
Processing ACCO
Processing ACDC
Processing ACEL
Processing ACER
Processing ACET
Processing ACGL

None

In [56]:
#window_df.to_csv('full_stock_history_5m.csv')

In [55]:
ticker_window_count = {}
ticker_start_window = {}

window_cal= nyse_cal.schedule(start_date=start_date, end_date=end_date)
valid_days = mcal.date_range(window_cal, frequency='5M')

one_day_size = valid_days.get_loc( valid_days[0] + pd.Timedelta("1D") ) 
segment_size = one_day_size * 2

cur_pos =  0
last_pos = len(valid_days) 

while cur_pos < last_pos:

    start_window = valid_days[ cur_pos ]
    end_window = valid_days[ min(cur_pos + segment_size, last_pos-1) ] 

    tickers_in_sub_window = get_tickers_for_window( start_window, end_window )

    print(f"Processing {start_window} to {end_window} with {len(tickers_in_sub_window)} tickers")

    sub_window = (window_df[start_window:end_window]).copy()

    columns_to_drop = [i for i in sub_window.columns if i not in tickers_in_sub_window]

    sub_window.drop(columns=columns_to_drop, inplace=True)
    
    sub_window_name = f"{start_window.strftime('%Y-%m-%d')}-{end_window.strftime('%Y-%m-%d')}"
    sub_window_file = f"./window_data/{sub_window_name}.csv"

    sub_window.to_csv(sub_window_file)

    for ticker in tickers_in_sub_window:
        if( ticker in ticker_window_count.keys() ):
            ticker_window_count[ticker] += 1
        else:
            ticker_window_count[ticker] = 1
            ticker_start_window[ticker] = sub_window_name

    cur_pos += segment_size


        # if( ticker in ticker_window_count.keys() ):
        #     ticker_window_count[ticker] += 1
        # else:
        #     ticker_window_count[ticker] = 1

        # if window_df is None:
        #     window_df = td
        # else:
        #     window_df = pd.concat([window_df, td], axis=1)




Processing 2006-01-03 14:35:00+00:00 to 2006-01-05 14:35:00+00:00 with 525 tickers
Processing 2006-01-05 14:35:00+00:00 to 2006-01-09 14:35:00+00:00 with 525 tickers
Processing 2006-01-09 14:35:00+00:00 to 2006-01-11 14:35:00+00:00 with 525 tickers
Processing 2006-01-11 14:35:00+00:00 to 2006-01-13 14:35:00+00:00 with 525 tickers
Processing 2006-01-13 14:35:00+00:00 to 2006-01-18 14:35:00+00:00 with 525 tickers
Processing 2006-01-18 14:35:00+00:00 to 2006-01-20 14:35:00+00:00 with 525 tickers
Processing 2006-01-20 14:35:00+00:00 to 2006-01-24 14:35:00+00:00 with 525 tickers
Processing 2006-01-24 14:35:00+00:00 to 2006-01-26 14:35:00+00:00 with 525 tickers
Processing 2006-01-26 14:35:00+00:00 to 2006-01-30 14:35:00+00:00 with 526 tickers
Processing 2006-01-30 14:35:00+00:00 to 2006-02-01 14:35:00+00:00 with 526 tickers
Processing 2006-02-01 14:35:00+00:00 to 2006-02-03 14:35:00+00:00 with 527 tickers
Processing 2006-02-03 14:35:00+00:00 to 2006-02-07 14:35:00+00:00 with 527 tickers
Proc

In [82]:
#
# Read segment files and run analysis
#
import re
from sklearn.neighbors import NearestNeighbors


swindow_list =  os.listdir("./window_data")
skipped = 0
matched = 0



for swindow in swindow_list:
    sdate = None
    edate = None



    regex = r"(\d{4}\-\d{2}\-\d{2})\-(\d{4}\-\d{2}\-\d{2}).csv$"
    matches = re.search(regex, swindow)
    if matches:
        sdate = pd.Timestamp(matches.group(1))
        edate = pd.Timestamp(matches.group(2))
       
        print(f"start_date: {sdate}, end_date: {edate}")
    else:
        print(f"Skipping {swindow}")
        skipped += 1
        continue

    tmp_df = pd.read_csv(f"./window_data/{swindow}", index_col="Date", parse_dates=True, infer_datetime_format=True)

    old_tmp = tmp_df.copy()

    x = tmp_df.isna().sum()
    cut_off = tmp_df.shape[0]*.05
    drop_tickers  = x[x > cut_off].index.tolist()

    print(f"Cut off is {cut_off} NA's -- {len(drop_tickers)}) tickers to drop")

    tmp_df.drop(columns=drop_tickers, inplace=True)


    tmp_learn_df = (tmp_df.pct_change()).dropna().transpose()*100

    neigh_list = NearestNeighbors(n_neighbors=25)
    neigh_list.fit(tmp_learn_df.values.copy())

    comp_df = pd.DataFrame(columns=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
                                    'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20'],index=tmp_learn_df.index.tolist())

# Set Ticker column as index
# Set 'C1'-'C20' and 'D1'-'D20' columns as float

    comp_df[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10',\
            'C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
            'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',\
            'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20']] = comp_df[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10',\
            'C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
            'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',\
            'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20']].astype(float)


                                    

    for index, row in tmp_learn_df.iterrows():
        dist, ind = neigh_list.kneighbors([row])
        #print(f"Ticker: {index}  Dist: {dist}  Ind: {ind}")

        row_comp_list = []
        row_distance_list = []
        for i in range(1,21):
            ccol = f"C{i}"
            dcol = f"D{i}"
            comp_ticker = tmp_learn_df.index[ind[0][i]]
            comp_distance = dist[0][i]

            comp_df.loc[index, ccol] = comp_ticker
            comp_df.loc[index, dcol] = comp_distance



    swindow_filename = Path( f"./comp_data/comps-{swindow}"  )
    print(f"Writing {swindow_filename}")
    comp_df.to_csv(swindow_filename)




    #dist[i], ind[i] = neigh_list[i].kneighbors([x])

    


start_date: 2020-02-11 00:00:00, end_date: 2020-02-13 00:00:00
Cut off is 7.8500000000000005 NA's -- 1685) tickers to drop
Writing comp_data/comps-2020-02-11-2020-02-13.csv


In [71]:
import numpy as np

comp_df = pd.DataFrame(np.nan, columns=['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
                                    'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12','D13','D14','D15','D16','D17','D18','D19','D20'], index = tickers_in_window)

# Set Ticker column as index
# Set 'C1'-'C20' and 'D1'-'D20' columns as float

comp_df[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10',\
            'C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
            'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',\
            'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20']] = comp_df[['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10',\
            'C11','C12','C13','C14','C15','C16','C17','C18','C19','C20',\
            'D1','D2','D3','D4','D5','D6','D7','D8','D9','D10',\
            'D11','D12','D13','D14','D15','D16','D17','D18','D19','D20']].astype(float)





In [ ]:
def build_stock_dataframe( tickers, start_date, end_date, saved=False ):
    outdf = None
    missing_tickers = []
    i=0

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    days_period = len( (nyse_cal.schedule(start_date=start_date, end_date=end_date)).index.unique())

    peroid_multiplier = 1

    if( urlarguments['timeframe'] == '1min'):
        peroid_multiplier = 250 #390   

    elif( urlarguments['timeframe'] == '5min'):
        peroid_multiplier = 65 #72
    elif( urlarguments['timeframe'] == '30min'):
        peroid_multiplier = 12      
    
    elif( urlarguments['timeframe'] == '1hour'):
        peroid_multiplier = 6

    elif( urlarguments['timeframe'] == '1day'):
        peroid_multiplier = 1
    else:
        print(f"Invalid timeframe {urlarguments['timeframe']}")
        return None

    print(f"Days Period {days_period} - Period Multiplier {peroid_multiplier}")

    for ticker in tickers.index:

        filename = Path(f"./tickdata/{ticker}_full_{urlarguments['timeframe']}_adjsplit.txt")  

        if os.path.exists(filename) == False:
            missing_tickers.append(ticker)
            continue
       

        stock_start = pd.to_datetime(first_ratetickers.loc[ticker]['start_date'])

        if(ticker == 'ABNB'):
            print(f"ANBB {ticker} start date {stock_start} is greater than the start date {start_date}")
  
  
        if( stock_start > start_date):
            print(f"Stock {ticker} start date {stock_start} is greater than the start date {start_date}")
            days_period_stock = len( (nyse_cal.schedule(start_date=stock_start, end_date=end_date)).index.unique())
      
        else:
            days_period_stock = days_period
 
        df = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True)

    
        df.columns = ['Date', 'Open', 'High', 'Low', ticker, 'Volume']
        df['Date']= pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)
        
        df_sel = df[ticker].loc[start_date:end_date].to_frame()
        df_sel = df_sel[~df_sel.index.duplicated()]
        df_sel.index = pd.to_datetime(df_sel.index)
        
        df_sel = df_sel.between_time('09:30', '16:00')

        expected = days_period_stock * peroid_multiplier 
        rec = len(df_sel)
        if(rec < expected):
            if( (expected/rec)>2):
                continue
            print(f"{days_period}:{urlarguments['timeframe']} Ticker {ticker} has {rec} records - expected {expected}")
            continue

        i = i + 1
        if( i % 100 == 0):
            print(f"Processed {i} tickers")

        #print(f"{i} {ticker} has {len(df_sel)} records")
        if outdf is None:
            outdf = df_sel
        else:
            outdf = pd.concat([outdf,df_sel], axis=1)

    #outdf.dropna(inplace=True)
    print(missing_tickers)
    return outdf



In [ ]:
#start_date = '2022-05-31' # 09:00:00'
start_date = '2018-01-01' # 09:00:00'
end_date = '2023-06-06' # 16:00:00'

load_data = True

#outdf = build_stock_dataframe(['NVDA','ILMN','MSFT','NVDA','TTD','XOM'], start_date, end_date )
if load_data==False:
    outdf = build_stock_dataframe(full_stock_list, start_date, end_date)
    outdf.to_csv("top_stocks_five_year-5m-1.csv")
else:
    outdf = pd.read_csv("top_stocks_five_year-5m-1.csv", index_col="Date", parse_dates=True, infer_datetime_format=True)
    

outdf.index = outdf.index.tz_localize(tz='US/Eastern')


In [ ]:
#outdf.to_csv("top_stocks_one_year.csv")
# outdf.tail()

# working_df = outdf.copy()
# x=  working_df.isna().sum()

valid_days

In [ ]:
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2023-06-06')


working_df = outdf.copy()
#x=  working_df.isna().sum()


print("Resample start")
#working_df = working_df.resample('60min').mean()
freq = pd.Timedelta(urlarguments['timeframe'])    
#working_df = working_df.resample(freq).mean()

print("Resample end")

print("Date start")
window_cal= nyse_cal.schedule(start_date=start_date, end_date=end_date)
valid_days = mcal.date_range(window_cal, frequency='5M')
working_df = working_df[ working_df.index.isin(valid_days) ]
print("Resample end")

print(working_df.shape)


#30min linimit
fill_limit = int( pd.Timedelta("30min") / pd.Timedelta(urlarguments['timeframe'] ))
working_df.fillna(method='ffill', limit=fill_limit, inplace=True)

# Calculate window size
window_size = len(valid_days)
one_day_size = valid_days.get_loc( (valid_days[0] + pd.Timedelta("1D")) ) 

print(working_df.shape)
x=  working_df.isna().sum()
cut_off = working_df.shape[0]*.01
drop_tickers  = x[x > cut_off].index.tolist()

print("Cut off is ", cut_off)

working_df.drop(columns=drop_tickers, inplace=True)

y=  working_df.isna().sum()


print("Dropped columns ")
# # display(working_df.head())
# # display(working_df.shape)

working_df = (working_df.pct_change()).dropna().transpose()*100
# # display(working_df.head())
# # display(working_df.shape)


# # def scale_data( x ):
# #     if x < 0:
# #         return -1
# #     elif x > 0:
# #         return 1
# #     else:
# #         return 0
    
# # #working_df = working_df.applymap(scale_data)
# # #working_df = working_df[working_df.columns[1500:]]

seg_size = int(one_day_size*2)
seg_count = int(working_df.shape[1] / seg_size)
seg_offset = 0
#seg_size = int( (working_df.shape[1]-seg_offset) / seg_count )

print("Seg count ", seg_count, " Seg size ", seg_size)
seg_df = [None]*(seg_count)
for i in range(0, seg_count):
    seg_df[i] = working_df.iloc[:,(seg_offset+i*seg_size):((seg_offset+(i+1)*seg_size))]

seg_df[0].shape


In [ ]:
x = seg_df[0]
y = seg_df[-1]
display(x.head())
display(y.head())
display(mcal.date_range(window_cal, frequency='1D').shape)

#end_slice = pd.to_datetime(start_date) + pd.Timedelta("30days")
#x= valid_days[1000]

In [ ]:
original = working_df.values.copy()

#scaled = StandardScaler().fit_transform(working_df.values.copy())
#scaled = MinMaxScaler().fit_transform(russell_tmp)

#scaled = working_df.values.copy()   

from sklearn.neighbors import NearestNeighbors

seg_count = len(seg_df)
neigh_list = [None]*seg_count

for i in range(0,seg_count):
    neigh_list[i] = NearestNeighbors(n_neighbors=50)
    neigh_list[i].fit(seg_df[i].values.copy())
    print(f"Segment {i} done")


In [ ]:
double_ticker = ['GOOG','GOOGL','NWS','NWSA','FOX', 'FOXA']

def get_comps( input_ticker, ignore_cutoff = False):
    xloc = working_df.index.get_loc(input_ticker)
    x = original[xloc]

    seg_count = len(seg_df)

    dist = [None]*seg_count
    ind = [None]*seg_count
    max_comps = 5
    returned_comps = {}
    returned_comps_distance = {}
    returned_comps_name = {}
    starting_comp = 1



    for i in range(0,seg_count):
        x = seg_df[i].iloc[xloc]
        dist[i], ind[i] = neigh_list[i].kneighbors([x])

    for i in range(0,seg_count):
        for j in range(starting_comp, starting_comp+ max_comps+1): #len(ind[0])):
            ticker = seg_df[i].index[ind[i][0][j]]
                                    

            if ticker in returned_comps.keys():
                returned_comps[ticker] = returned_comps[ticker] + 1
                returned_comps_distance[ticker] = returned_comps_distance[ticker] + dist[i][0][j]
            else:
                returned_comps_name[ticker] = first_ratetickers.loc[ticker]['Name']
                returned_comps[ticker] = 1
                returned_comps_distance[ticker] = dist[i][0][j]




    returned_comps = {k: v for k, v in sorted(returned_comps.items(), reverse=True, key=lambda item: item[1])}

    rc_df = pd.DataFrame.from_dict(returned_comps, orient='index', columns=['Window'])
    rcd_df = pd.DataFrame.from_dict(returned_comps_distance, orient='index', columns=['TDistance'])
    rcn_df =  pd.DataFrame.from_dict(returned_comps_name, orient='index', columns=['Name'])

    comps_df = pd.concat([rcn_df,rc_df,rcd_df], axis=1)

    
    comps_df.sort_values(by=['Window'], inplace=True, ascending=False)
    if input_ticker in double_ticker:
        comps_df.drop( comps_df.index[0],inplace=True)


    comps_df['AvgDistance'] = comps_df['TDistance']/comps_df['Window']
    
    comps_df['NetSort'] = comps_df['Window']/comps_df['AvgDistance']


    comps_df.drop( comps_df.index[ comps_df['Window']  == 1],inplace=True)

    if ignore_cutoff == False:
        cut_off = rc_df['Window'].mean() + (rc_df['Window'].std())
    else:
        cut_off = 1 #rc_df['Window'].quantile(.10 )
                                    
    

    comps_df.drop( comps_df.index[ comps_df['Window'] < cut_off],inplace=True)


    # Clear comps in window ranking
    if ( ((comps_df['Window'].max()/comps_df["Window"].min()) >= 2)  ):
        #cut_off = min( seg_count/2, comps_df['Window'].quantile(.35))
        cut_off = comps_df["Window"].max() - (comps_df['Window'].std()*2.2)

        print(f"Cut off is {cut_off}")
        if(ignore_cutoff == False):
            comps_df.drop( comps_df.index[ comps_df['Window'] < cut_off],inplace=True)
             

        if( comps_df["Window"].max() < (seg_count*.2)) and ignore_cutoff == False:
                comps_df.sort_values(by=['AvgDistance'], inplace=True, ascending=True)
        else:
            comps_df.sort_values(by=['Window'], inplace=True, ascending=False)
    else:
        comps_df.sort_values(by=['AvgDistance'], inplace=True, ascending=True)



    #comps_df.head(20)

    # i=0
    # for ticker in returned_comps:
    #     i = i + 1
    #     company_name = first_ratetickers.loc[ticker]['Name']
    #     print(i,ticker,company_name, returned_comps[ticker], round(returned_comps_distance[ticker]/returned_comps[ticker],2))

    #display(comps_df)
    return comps_df

In [ ]:
comps_df = get_comps("FOX",True)

display(comps_df.head(20))




In [ ]:
display(msft[140:175])